In [7]:
import os
print(os.getcwd())

d:\GithubRepos\PIMCO-Text2SQL


In [8]:
import sys
import os

import re
import compare_csv

In [9]:
text = """
1. "List the top 5 registrants by total net assets, including their CIK and country."
   SQL: 
   WITH FundAssets AS (
       SELECT R.CIK, R.REGISTRANT_NAME, R.COUNTRY, F.NET_ASSETS
       FROM REGISTRANT R
       JOIN FUND_REPORTED_INFO F ON R.ACCESSION_NUMBER = F.ACCESSION_NUMBER
   )
   SELECT CIK, REGISTRANT_NAME, COUNTRY, NET_ASSETS
   FROM FundAssets
   ORDER BY NET_ASSETS DESC
   LIMIT 5;

2. "Find all holdings with a fair value level of Level 1 and their corresponding fund names."
   SQL: 
   WITH HoldingsCTE AS (
       SELECT H.HOLDING_ID, H.ISSUER_NAME, H.FAIR_VALUE_LEVEL, F.SERIES_NAME
       FROM FUND_REPORTED_HOLDING H
       JOIN FUND_REPORTED_INFO F ON H.ACCESSION_NUMBER = F.ACCESSION_NUMBER
       WHERE H.FAIR_VALUE_LEVEL = 'Level 1'
   )
   SELECT HOLDING_ID, ISSUER_NAME, SERIES_NAME
   FROM HoldingsCTE;

3. "Calculate the total collateral amount for repurchase agreements grouped by counterparty."
   SQL: 
   WITH CollateralCTE AS (
    SELECT RCP.NAME AS Counterparty_Name, SUM(RC.COLLATERAL_AMOUNT) AS Total_Collateral
    FROM REPURCHASE_COLLATERAL RC
    JOIN REPURCHASE_COUNTERPARTY RCP ON RC.HOLDING_ID = RCP.HOLDING_ID
    GROUP BY RCP.NAME
   )
   SELECT Counterparty_Name, Total_Collateral
   FROM CollateralCTE
   ORDER BY Total_Collateral DESC;

4. "Locate funds that have both securities lending activities and repurchase agreements."
   SQL: 
   WITH SecuritiesLending AS (
       SELECT ACCESSION_NUMBER
       FROM SECURITIES_LENDING
       WHERE IS_LOAN_BY_FUND = 'Y'
   ),
   RepurchaseAgreements AS (
       SELECT ACCESSION_NUMBER
       FROM REPURCHASE_AGREEMENT
   )
   SELECT F.SERIES_NAME
   FROM FUND_REPORTED_INFO F
   WHERE F.ACCESSION_NUMBER IN (SELECT ACCESSION_NUMBER FROM SecuritiesLending)
     AND F.ACCESSION_NUMBER IN (SELECT ACCESSION_NUMBER FROM RepurchaseAgreements);

5. "Find borrowers who have borrowed more than $5,000,000, including their names and LEIs."
   SQL: 
   WITH BorrowedAmounts AS (
       SELECT BORROWER_ID, SUM(AGGREGATE_VALUE) AS Total_Borrowed
       FROM BORROWER
       GROUP BY BORROWER_ID
       HAVING SUM(AGGREGATE_VALUE) > 5000000
   )
   SELECT B.NAME, B.LEI, BA.Total_Borrowed
   FROM BORROWER B
   JOIN BorrowedAmounts BA ON B.BORROWER_ID = BA.BORROWER_ID;

6. "List all derivative counterparties along with the number of derivative instruments they are involved in."
   SQL: 
   WITH CounterpartyCounts AS (
       SELECT DC.DERIVATIVE_COUNTERPARTY_NAME, COUNT(*) AS Instrument_Count
       FROM DERIVATIVE_COUNTERPARTY DC
       JOIN FUND_REPORTED_HOLDING H ON DC.HOLDING_ID = H.HOLDING_ID
       JOIN DEBT_SECURITY D ON H.HOLDING_ID = D.HOLDING_ID
       GROUP BY DC.DERIVATIVE_COUNTERPARTY_NAME
   )
   SELECT DERIVATIVE_COUNTERPARTY_NAME, Instrument_Count
   FROM CounterpartyCounts
   ORDER BY Instrument_Count DESC;

7. "Compute the average annualized rate for debt securities grouped by coupon type."
   SQL: 
   WITH RateAverages AS (
       SELECT DS.COUPON_TYPE, AVG(DS.ANNUALIZED_RATE) AS Avg_Annualized_Rate
       FROM DEBT_SECURITY DS
       WHERE DS.ANNUALIZED_RATE IS NOT NULL
       GROUP BY DS.COUPON_TYPE
   )
   SELECT COUPON_TYPE, Avg_Annualized_Rate
   FROM RateAverages
   ORDER BY Avg_Annualized_Rate DESC;

8. "Get funds that have experienced a net decrease in assets over the last three reporting periods."
   SQL: 
   WITH AssetChanges AS (
       SELECT F.ACCESSION_NUMBER, F.SERIES_NAME, S.REPORT_DATE, F.NET_ASSETS,
              LAG(F.NET_ASSETS, 1) OVER (PARTITION BY F.SERIES_NAME ORDER BY S.REPORT_DATE) AS Previous_Period_Assets
       FROM FUND_REPORTED_INFO F
       JOIN SUBMISSION S ON F.ACCESSION_NUMBER = S.ACCESSION_NUMBER
   )
   SELECT DISTINCT AC.SERIES_NAME
   FROM AssetChanges AC
   WHERE AC.NET_ASSETS < AC.Previous_Period_Assets
     AND AC.Previous_Period_Assets IS NOT NULL;

9. "Identify issuers with more than three different securities holdings, including their names and CUSIPs."
   SQL: 
   WITH IssuerHoldings AS (
       SELECT H.ISSUER_NAME, H.ISSUER_CUSIP, COUNT(DISTINCT H.HOLDING_ID) AS Holding_Count
       FROM FUND_REPORTED_HOLDING H
       GROUP BY H.ISSUER_NAME, H.ISSUER_CUSIP
       HAVING COUNT(DISTINCT H.HOLDING_ID) > 3
   )
   SELECT ISSUER_NAME, ISSUER_CUSIP, Holding_Count
   FROM IssuerHoldings
   ORDER BY Holding_Count DESC;

10. "Calculate the total notional amount of derivatives per currency and identify the top 3 currencies by notional amount."
    SQL: 
    WITH NotionalSums AS (
        SELECT ODNA.CURRENCY_CODE, SUM(ODNA.NOTIONAL_AMOUNT) AS Total_Notional
        FROM OTHER_DERIV_NOTIONAL_AMOUNT ODNA
        GROUP BY ODNA.CURRENCY_CODE
    )
    SELECT CURRENCY_CODE, Total_Notional
    FROM NotionalSums
    ORDER BY Total_Notional DESC
    LIMIT 3;

11. "List funds with liquidation preferences exceeding their net assets."
    SQL: 
    WITH FundPreferences AS (
        SELECT F.SERIES_NAME, F.LIQUIDATION_PREFERENCE, F.NET_ASSETS
        FROM FUND_REPORTED_INFO F
    )
    SELECT SERIES_NAME, LIQUIDATION_PREFERENCE, NET_ASSETS
    FROM FundPreferences
    WHERE LIQUIDATION_PREFERENCE > NET_ASSETS;

12. "Find all convertible securities that are contingent and have a conversion ratio above 1.5."
    SQL: 
    WITH ConvertibleCTE AS (
        SELECT DS.HOLDING_ID, CSC.CONVERSION_RATIO
        FROM DEBT_SECURITY DS
        JOIN CONVERTIBLE_SECURITY_CURRENCY CSC ON DS.HOLDING_ID = CSC.HOLDING_ID
        WHERE DS.IS_CONVTIBLE_CONTINGENT = 'Y' AND CSC.CONVERSION_RATIO > 1.5
    )
    SELECT HOLDING_ID, CONVERSION_RATIO
    FROM ConvertibleCTE;

13. "Analyze the distribution of asset categories within the top 10 largest funds by total assets."
    SQL: 
    WITH TopFunds AS (
        SELECT SERIES_NAME, ACCESSION_NUMBER
        FROM FUND_REPORTED_INFO
        ORDER BY TOTAL_ASSETS DESC
        LIMIT 10
    ),
    AssetDistribution AS (
        SELECT H.ASSET_CAT, COUNT(*) AS Category_Count
        FROM FUND_REPORTED_HOLDING H
        JOIN TopFunds T ON H.ACCESSION_NUMBER = T.ACCESSION_NUMBER
        GROUP BY H.ASSET_CAT
    )
    SELECT ASSET_CAT, Category_Count
    FROM AssetDistribution
    ORDER BY Category_Count DESC;
    
14. "Find the top 10 funds with the highest average monthly returns in the past quarter."
   SQL: 
   WITH AvgMonthlyReturn AS (
       SELECT ACCESSION_NUMBER, 
              (MONTHLY_TOTAL_RETURN1 + MONTHLY_TOTAL_RETURN2 + MONTHLY_TOTAL_RETURN3) / 3.0 AS Avg_Return
       FROM MONTHLY_TOTAL_RETURN
   )
   SELECT F.SERIES_NAME, A.ACCESSION_NUMBER, A.Avg_Return
   FROM AvgMonthlyReturn A
   JOIN FUND_REPORTED_INFO F ON A.ACCESSION_NUMBER = F.ACCESSION_NUMBER
   ORDER BY A.Avg_Return DESC
   LIMIT 10;

15. "Compare the latest net asset values of the top 5 performing funds."
    SQL: 
    WITH TopPerformingFunds AS (
        SELECT 
            ACCESSION_NUMBER, 
            (MONTHLY_TOTAL_RETURN1 + MONTHLY_TOTAL_RETURN2 + MONTHLY_TOTAL_RETURN3) / 3.0 AS Avg_Return
        FROM 
            MONTHLY_TOTAL_RETURN
        ORDER BY 
            Avg_Return DESC
        LIMIT 5
    )
    SELECT 
        FR.SERIES_NAME, 
        FR.NET_ASSETS, 
        TP.Avg_Return
    FROM 
        TopPerformingFunds TP
    JOIN 
        FUND_REPORTED_INFO FR ON TP.ACCESSION_NUMBER = FR.ACCESSION_NUMBER;

16. "Calculate the overall average return across all funds for the most recent month."
    SQL: 
    WITH LatestReturns AS (
        SELECT 
            M.ACCESSION_NUMBER, 
            M.MONTHLY_TOTAL_RETURN1
        FROM 
            MONTHLY_TOTAL_RETURN M
        JOIN 
            SUBMISSION S ON M.ACCESSION_NUMBER = S.ACCESSION_NUMBER
        WHERE 
            S.REPORT_DATE = (SELECT MAX(REPORT_DATE) FROM SUBMISSION)
    )
    SELECT 
        AVG(MONTHLY_TOTAL_RETURN1) AS Average_Return
    FROM 
        LatestReturns;

17. "Find the interest rate risk for each fund and identify those with the highest risk scores."
    SQL: 
    WITH InterestRiskScores AS (
        SELECT 
            IR.ACCESSION_NUMBER, 
            -- Calculating composite risk score by summing absolute values of DV01 and DV100 columns
            (ABS(CAST(IR.INTRST_RATE_CHANGE_3MON_DV01 AS FLOAT)) +
            ABS(CAST(IR.INTRST_RATE_CHANGE_1YR_DV01 AS FLOAT)) +
            ABS(CAST(IR.INTRST_RATE_CHANGE_5YR_DV01 AS FLOAT)) +
            ABS(CAST(IR.INTRST_RATE_CHANGE_10YR_DV01 AS FLOAT)) +
            ABS(CAST(IR.INTRST_RATE_CHANGE_30YR_DV01 AS FLOAT)) +
            ABS(CAST(IR.INTRST_RATE_CHANGE_3MON_DV100 AS FLOAT)) +
            ABS(CAST(IR.INTRST_RATE_CHANGE_1YR_DV100 AS FLOAT)) +
            ABS(CAST(IR.INTRST_RATE_CHANGE_5YR_DV100 AS FLOAT)) +
            ABS(CAST(IR.INTRST_RATE_CHANGE_10YR_DV100 AS FLOAT)) +
            ABS(CAST(IR.INTRST_RATE_CHANGE_30YR_DV100 AS FLOAT))
            ) AS Composite_Risk_Score
        FROM 
            INTEREST_RATE_RISK IR
    )
    SELECT 
        FR.SERIES_NAME, 
        FR.ACCESSION_NUMBER, 
        IRS.Composite_Risk_Score
    FROM 
        InterestRiskScores IRS
    JOIN 
        FUND_REPORTED_INFO FR ON IRS.ACCESSION_NUMBER = FR.ACCESSION_NUMBER
    ORDER BY 
        IRS.Composite_Risk_Score DESC
    LIMIT 5;

18. "Analyze the composition of fund portfolios by categorizing assets and their total values."
    SQL: 
    WITH PortfolioComposition AS (
    SELECT 
        ACCESSION_NUMBER, 
        ASSET_CAT, 
        SUM(CAST(CURRENCY_VALUE AS FLOAT)) AS Total_Value
    FROM 
        FUND_REPORTED_HOLDING
    GROUP BY 
        ACCESSION_NUMBER, 
        ASSET_CAT
    )
    SELECT 
        F.SERIES_NAME, 
        PC.ASSET_CAT, 
        PC.Total_Value
    FROM 
        PortfolioComposition PC
    JOIN 
        FUND_REPORTED_INFO F ON PC.ACCESSION_NUMBER = F.ACCESSION_NUMBER
    ORDER BY 
        F.SERIES_NAME, 
        PC.Total_Value DESC;

19. "Identify the most common asset categories across all fund portfolios."
    SQL: 
    WITH AssetCounts AS (
        SELECT ASSET_CAT, COUNT(*) AS Count
        FROM FUND_REPORTED_HOLDING
        GROUP BY ASSET_CAT
    )
    SELECT ASSET_CAT, Count
    FROM AssetCounts
    ORDER BY Count DESC
    LIMIT 5;

20. "Retrieve funds that have experienced a net decrease in assets over the last three reporting periods."
   SQL: 
   WITH AssetChanges AS (
       SELECT F.ACCESSION_NUMBER, F.SERIES_NAME, S.REPORT_DATE, F.NET_ASSETS,
              LAG(F.NET_ASSETS, 1) OVER (PARTITION BY F.SERIES_NAME ORDER BY S.REPORT_DATE) AS Previous_Period_Assets
       FROM FUND_REPORTED_INFO F
       JOIN SUBMISSION S ON F.ACCESSION_NUMBER = S.ACCESSION_NUMBER
   )
   SELECT DISTINCT AC.SERIES_NAME
   FROM AssetChanges AC
   WHERE AC.NET_ASSETS < AC.Previous_Period_Assets
     AND AC.Previous_Period_Assets IS NOT NULL;"""



In [10]:
pattern = r'"\s*(.*?)\s*"\s*SQL:\s*(WITH.*?;)(?=\n\s*\d+|$)'
matches = re.findall(pattern, text, re.DOTALL)


In [11]:
ground_truth_query = [match[1] for match in matches]
llm_query = [match[0] for match in matches]
print("Queries:", ground_truth_query)
print("SQL Statements:", llm_query)

Queries: ['WITH FundAssets AS (\n       SELECT R.CIK, R.REGISTRANT_NAME, R.COUNTRY, F.NET_ASSETS\n       FROM REGISTRANT R\n       JOIN FUND_REPORTED_INFO F ON R.ACCESSION_NUMBER = F.ACCESSION_NUMBER\n   )\n   SELECT CIK, REGISTRANT_NAME, COUNTRY, NET_ASSETS\n   FROM FundAssets\n   ORDER BY NET_ASSETS DESC\n   LIMIT 5;', "WITH HoldingsCTE AS (\n       SELECT H.HOLDING_ID, H.ISSUER_NAME, H.FAIR_VALUE_LEVEL, F.SERIES_NAME\n       FROM FUND_REPORTED_HOLDING H\n       JOIN FUND_REPORTED_INFO F ON H.ACCESSION_NUMBER = F.ACCESSION_NUMBER\n       WHERE H.FAIR_VALUE_LEVEL = 'Level 1'\n   )\n   SELECT HOLDING_ID, ISSUER_NAME, SERIES_NAME\n   FROM HoldingsCTE;", 'WITH CollateralCTE AS (\n    SELECT RCP.NAME AS Counterparty_Name, SUM(RC.COLLATERAL_AMOUNT) AS Total_Collateral\n    FROM REPURCHASE_COLLATERAL RC\n    JOIN REPURCHASE_COUNTERPARTY RCP ON RC.HOLDING_ID = RCP.HOLDING_ID\n    GROUP BY RCP.NAME\n   )\n   SELECT Counterparty_Name, Total_Collateral\n   FROM CollateralCTE\n   ORDER BY Total_

In [12]:
results = []
while len(results) < 20:
    results.append(None)
print(len(results))
print(len(llm_query))

print("Are the following queries the same?")
i=0
print("i = ", i)


20
20
Are the following queries the same?
i =  0


In [13]:
#for i in range(len(llm_query)):
    #results[i]=(str(i)+'. '+ str(compare_csv.compare_csv(ground_truth_query[i],llm_query[i])))

In [14]:
if i<len(llm_query):
    results[i]=(str(i)+'. '+ str(compare_csv.compare_csv(ground_truth_query[i],llm_query[i])))
i+=1 #1
print("i = ", i)


2024-11-05 10:57:16,390 - compare_csv - INFO - Executing LLM query
2024-11-05 10:57:16,393 - urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): localhost:8000
2024-11-05 10:57:20,129 - urllib3.connectionpool - DEBUG - http://localhost:8000 "POST /query HTTP/11" 200 721
2024-11-05 10:57:20,130 - compare_csv - INFO - Executing grouth truth query...
2024-11-05 10:57:20,130 - chatgpt_api.api - DEBUG - Executing SQL query: WITH FundAssets AS (
       SELECT R.CIK, R.REGISTRANT_NAME, R.COUNTRY, F.NET_ASSETS
       FROM REGISTRANT R
       JOIN FUND_REPORTED_INFO F ON R.ACCESSION_NUMBER = F.ACCESSION_NUMBER
   )
   SELECT CIK, REGISTRANT_NAME, COUNTRY, NET_ASSETS
   FROM FundAssets
   ORDER BY NET_ASSETS DESC
   LIMIT 5;
2024-11-05 10:57:20,143 - chatgpt_api.api - INFO - SQL query executed successfully


CSV outputs match perfectly.
i =  1


In [15]:
if i<len(llm_query):
    results[i]=(str(i)+'. '+ str(compare_csv.compare_csv(ground_truth_query[i],llm_query[i])))
i+=1 #2
print("i = ", i)


2024-11-05 10:57:20,152 - compare_csv - INFO - Executing LLM query
2024-11-05 10:57:20,154 - urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): localhost:8000
2024-11-05 10:57:23,384 - urllib3.connectionpool - DEBUG - http://localhost:8000 "POST /query HTTP/11" 200 307
2024-11-05 10:57:23,385 - compare_csv - INFO - Executing grouth truth query...
2024-11-05 10:57:23,385 - chatgpt_api.api - DEBUG - Executing SQL query: WITH HoldingsCTE AS (
       SELECT H.HOLDING_ID, H.ISSUER_NAME, H.FAIR_VALUE_LEVEL, F.SERIES_NAME
       FROM FUND_REPORTED_HOLDING H
       JOIN FUND_REPORTED_INFO F ON H.ACCESSION_NUMBER = F.ACCESSION_NUMBER
       WHERE H.FAIR_VALUE_LEVEL = 'Level 1'
   )
   SELECT HOLDING_ID, ISSUER_NAME, SERIES_NAME
   FROM HoldingsCTE;
2024-11-05 10:57:23,390 - chatgpt_api.api - INFO - SQL query executed successfully


Mismatch found.
i =  2


In [16]:
if i<len(llm_query):
    results[i]=(str(i)+'. '+ str(compare_csv.compare_csv(ground_truth_query[i],llm_query[i])))
i+=1 #3
print("i = ", i)

2024-11-05 10:57:23,404 - compare_csv - INFO - Executing LLM query
2024-11-05 10:57:23,407 - urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): localhost:8000
2024-11-05 10:57:27,211 - urllib3.connectionpool - DEBUG - http://localhost:8000 "POST /query HTTP/11" 200 588
2024-11-05 10:57:27,212 - compare_csv - INFO - Executing grouth truth query...
2024-11-05 10:57:27,212 - chatgpt_api.api - DEBUG - Executing SQL query: WITH CollateralCTE AS (
    SELECT RCP.NAME AS Counterparty_Name, SUM(RC.COLLATERAL_AMOUNT) AS Total_Collateral
    FROM REPURCHASE_COLLATERAL RC
    JOIN REPURCHASE_COUNTERPARTY RCP ON RC.HOLDING_ID = RCP.HOLDING_ID
    GROUP BY RCP.NAME
   )
   SELECT Counterparty_Name, Total_Collateral
   FROM CollateralCTE
   ORDER BY Total_Collateral DESC;
2024-11-05 10:57:27,213 - chatgpt_api.api - INFO - SQL query executed successfully


CSV outputs match perfectly.
i =  3


In [17]:
if i<len(llm_query):
    results[i]=(str(i)+'. '+ str(compare_csv.compare_csv(ground_truth_query[i],llm_query[i])))
i+=1 #4
print("i = ", i)

2024-11-05 10:57:27,221 - compare_csv - INFO - Executing LLM query
2024-11-05 10:57:27,223 - urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): localhost:8000
2024-11-05 10:57:32,560 - urllib3.connectionpool - DEBUG - http://localhost:8000 "POST /query HTTP/11" 200 109526
2024-11-05 10:57:32,562 - compare_csv - INFO - Executing grouth truth query...
2024-11-05 10:57:32,562 - chatgpt_api.api - DEBUG - Executing SQL query: WITH SecuritiesLending AS (
       SELECT ACCESSION_NUMBER
       FROM SECURITIES_LENDING
       WHERE IS_LOAN_BY_FUND = 'Y'
   ),
   RepurchaseAgreements AS (
       SELECT ACCESSION_NUMBER
       FROM REPURCHASE_AGREEMENT
   )
   SELECT F.SERIES_NAME
   FROM FUND_REPORTED_INFO F
   WHERE F.ACCESSION_NUMBER IN (SELECT ACCESSION_NUMBER FROM SecuritiesLending)
     AND F.ACCESSION_NUMBER IN (SELECT ACCESSION_NUMBER FROM RepurchaseAgreements);
2024-11-05 10:57:34,399 - chatgpt_api.api - INFO - SQL query executed successfully


CSV outputs match perfectly.
i =  4


In [18]:
if i<len(llm_query):
    results[i]=(str(i)+'. '+ str(compare_csv.compare_csv(ground_truth_query[i],llm_query[i])))
i+=1 #5
print("i = ", i)

2024-11-05 10:57:34,411 - compare_csv - INFO - Executing LLM query
2024-11-05 10:57:34,414 - urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): localhost:8000
2024-11-05 10:57:40,002 - urllib3.connectionpool - DEBUG - http://localhost:8000 "POST /query HTTP/11" 200 168711
2024-11-05 10:57:40,004 - compare_csv - INFO - Executing grouth truth query...
2024-11-05 10:57:40,004 - chatgpt_api.api - DEBUG - Executing SQL query: WITH BorrowedAmounts AS (
       SELECT BORROWER_ID, SUM(AGGREGATE_VALUE) AS Total_Borrowed
       FROM BORROWER
       GROUP BY BORROWER_ID
       HAVING SUM(AGGREGATE_VALUE) > 5000000
   )
   SELECT B.NAME, B.LEI, BA.Total_Borrowed
   FROM BORROWER B
   JOIN BorrowedAmounts BA ON B.BORROWER_ID = BA.BORROWER_ID;
2024-11-05 10:57:40,045 - chatgpt_api.api - INFO - SQL query executed successfully


CSV outputs match perfectly.
i =  5


In [19]:
if i<len(llm_query):
    results[i]=(str(i)+'. '+ str(compare_csv.compare_csv(ground_truth_query[i],llm_query[i])))
i+=1 #6
print("i = ", i)

2024-11-05 10:57:40,059 - compare_csv - INFO - Executing LLM query
2024-11-05 10:57:40,062 - urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): localhost:8000
2024-11-05 10:57:44,666 - urllib3.connectionpool - DEBUG - http://localhost:8000 "POST /query HTTP/11" 200 2511
2024-11-05 10:57:44,667 - compare_csv - INFO - Executing grouth truth query...
2024-11-05 10:57:44,667 - chatgpt_api.api - DEBUG - Executing SQL query: WITH CounterpartyCounts AS (
       SELECT DC.DERIVATIVE_COUNTERPARTY_NAME, COUNT(*) AS Instrument_Count
       FROM DERIVATIVE_COUNTERPARTY DC
       JOIN FUND_REPORTED_HOLDING H ON DC.HOLDING_ID = H.HOLDING_ID
       JOIN DEBT_SECURITY D ON H.HOLDING_ID = D.HOLDING_ID
       GROUP BY DC.DERIVATIVE_COUNTERPARTY_NAME
   )
   SELECT DERIVATIVE_COUNTERPARTY_NAME, Instrument_Count
   FROM CounterpartyCounts
   ORDER BY Instrument_Count DESC;
2024-11-05 10:57:44,681 - chatgpt_api.api - INFO - SQL query executed successfully


Mismatch found.
i =  6


In [20]:
if i<len(llm_query):
    results[i]=(str(i)+'. '+ str(compare_csv.compare_csv(ground_truth_query[i],llm_query[i])))
i+=1 #7
print("i = ", i)

2024-11-05 10:57:44,690 - compare_csv - INFO - Executing LLM query
2024-11-05 10:57:44,691 - urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): localhost:8000
2024-11-05 10:57:49,158 - urllib3.connectionpool - DEBUG - http://localhost:8000 "POST /query HTTP/11" 200 553
2024-11-05 10:57:49,159 - compare_csv - INFO - Executing grouth truth query...
2024-11-05 10:57:49,159 - chatgpt_api.api - DEBUG - Executing SQL query: WITH RateAverages AS (
       SELECT DS.COUPON_TYPE, AVG(DS.ANNUALIZED_RATE) AS Avg_Annualized_Rate
       FROM DEBT_SECURITY DS
       WHERE DS.ANNUALIZED_RATE IS NOT NULL
       GROUP BY DS.COUPON_TYPE
   )
   SELECT COUPON_TYPE, Avg_Annualized_Rate
   FROM RateAverages
   ORDER BY Avg_Annualized_Rate DESC;
2024-11-05 10:57:49,163 - chatgpt_api.api - INFO - SQL query executed successfully


CSV outputs match perfectly.
i =  7


In [21]:
if i<len(llm_query):
    results[i]=(str(i)+'. '+ str(compare_csv.compare_csv(ground_truth_query[i],llm_query[i])))
i+=1 #8
print("i = ", i)

2024-11-05 10:57:49,172 - compare_csv - INFO - Executing LLM query
2024-11-05 10:57:49,175 - urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): localhost:8000
2024-11-05 10:58:01,823 - urllib3.connectionpool - DEBUG - http://localhost:8000 "POST /query HTTP/11" 200 1069
2024-11-05 10:58:01,823 - compare_csv - INFO - Executing grouth truth query...
2024-11-05 10:58:01,824 - chatgpt_api.api - DEBUG - Executing SQL query: WITH AssetChanges AS (
       SELECT F.ACCESSION_NUMBER, F.SERIES_NAME, S.REPORT_DATE, F.NET_ASSETS,
              LAG(F.NET_ASSETS, 1) OVER (PARTITION BY F.SERIES_NAME ORDER BY S.REPORT_DATE) AS Previous_Period_Assets
       FROM FUND_REPORTED_INFO F
       JOIN SUBMISSION S ON F.ACCESSION_NUMBER = S.ACCESSION_NUMBER
   )
   SELECT DISTINCT AC.SERIES_NAME
   FROM AssetChanges AC
   WHERE AC.NET_ASSETS < AC.Previous_Period_Assets
     AND AC.Previous_Period_Assets IS NOT NULL;
2024-11-05 10:58:01,841 - chatgpt_api.api - INFO - SQL query executed successfu

CSV outputs match perfectly.
i =  8


In [22]:
if i<len(llm_query):
    results[i]=(str(i)+'. '+ str(compare_csv.compare_csv(ground_truth_query[i],llm_query[i])))
i+=1 #9
print("i = ", i)

2024-11-05 10:58:01,852 - compare_csv - INFO - Executing LLM query
2024-11-05 10:58:01,854 - urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): localhost:8000
2024-11-05 10:58:14,565 - urllib3.connectionpool - DEBUG - http://localhost:8000 "POST /query HTTP/11" 200 13077
2024-11-05 10:58:14,566 - compare_csv - INFO - Executing grouth truth query...
2024-11-05 10:58:14,567 - chatgpt_api.api - DEBUG - Executing SQL query: WITH IssuerHoldings AS (
       SELECT H.ISSUER_NAME, H.ISSUER_CUSIP, COUNT(DISTINCT H.HOLDING_ID) AS Holding_Count
       FROM FUND_REPORTED_HOLDING H
       GROUP BY H.ISSUER_NAME, H.ISSUER_CUSIP
       HAVING COUNT(DISTINCT H.HOLDING_ID) > 3
   )
   SELECT ISSUER_NAME, ISSUER_CUSIP, Holding_Count
   FROM IssuerHoldings
   ORDER BY Holding_Count DESC;
2024-11-05 10:58:14,578 - chatgpt_api.api - INFO - SQL query executed successfully


CSV outputs match perfectly.
i =  9


In [23]:
if i<len(llm_query):
    results[i]=(str(i)+'. '+ str(compare_csv.compare_csv(ground_truth_query[i],llm_query[i])))
i+=1 #10
print("i = ", i)

2024-11-05 10:58:14,587 - compare_csv - INFO - Executing LLM query
2024-11-05 10:58:14,589 - urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): localhost:8000
2024-11-05 10:58:26,627 - urllib3.connectionpool - DEBUG - http://localhost:8000 "POST /query HTTP/11" 200 403
2024-11-05 10:58:26,628 - compare_csv - INFO - Executing grouth truth query...
2024-11-05 10:58:26,628 - chatgpt_api.api - DEBUG - Executing SQL query: WITH NotionalSums AS (
        SELECT ODNA.CURRENCY_CODE, SUM(ODNA.NOTIONAL_AMOUNT) AS Total_Notional
        FROM OTHER_DERIV_NOTIONAL_AMOUNT ODNA
        GROUP BY ODNA.CURRENCY_CODE
    )
    SELECT CURRENCY_CODE, Total_Notional
    FROM NotionalSums
    ORDER BY Total_Notional DESC
    LIMIT 3;
2024-11-05 10:58:26,629 - chatgpt_api.api - INFO - SQL query executed successfully


CSV outputs match perfectly.
i =  10


In [24]:
if i<len(llm_query):
    results[i]=(str(i)+'. '+ str(compare_csv.compare_csv(ground_truth_query[i],llm_query[i])))
i+=1 #11
print("i = ", i)

2024-11-05 10:58:26,638 - compare_csv - INFO - Executing LLM query
2024-11-05 10:58:26,642 - urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): localhost:8000
2024-11-05 10:58:39,978 - urllib3.connectionpool - DEBUG - http://localhost:8000 "POST /query HTTP/11" 200 1562
2024-11-05 10:58:39,979 - compare_csv - INFO - Executing grouth truth query...
2024-11-05 10:58:39,979 - chatgpt_api.api - DEBUG - Executing SQL query: WITH FundPreferences AS (
        SELECT F.SERIES_NAME, F.LIQUIDATION_PREFERENCE, F.NET_ASSETS
        FROM FUND_REPORTED_INFO F
    )
    SELECT SERIES_NAME, LIQUIDATION_PREFERENCE, NET_ASSETS
    FROM FundPreferences
    WHERE LIQUIDATION_PREFERENCE > NET_ASSETS;
2024-11-05 10:58:39,983 - chatgpt_api.api - INFO - SQL query executed successfully


CSV outputs match perfectly.
i =  11


In [25]:
if i<len(llm_query):
    results[i]=(str(i)+'. '+ str(compare_csv.compare_csv(ground_truth_query[i],llm_query[i])))
i+=1 #12
print("i = ", i)

2024-11-05 10:58:39,992 - compare_csv - INFO - Executing LLM query
2024-11-05 10:58:39,994 - urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): localhost:8000
2024-11-05 10:58:52,741 - urllib3.connectionpool - DEBUG - http://localhost:8000 "POST /query HTTP/11" 200 746
2024-11-05 10:58:52,742 - compare_csv - INFO - Executing grouth truth query...
2024-11-05 10:58:52,742 - chatgpt_api.api - DEBUG - Executing SQL query: WITH ConvertibleCTE AS (
        SELECT DS.HOLDING_ID, CSC.CONVERSION_RATIO
        FROM DEBT_SECURITY DS
        JOIN CONVERTIBLE_SECURITY_CURRENCY CSC ON DS.HOLDING_ID = CSC.HOLDING_ID
        WHERE DS.IS_CONVTIBLE_CONTINGENT = 'Y' AND CSC.CONVERSION_RATIO > 1.5
    )
    SELECT HOLDING_ID, CONVERSION_RATIO
    FROM ConvertibleCTE;
2024-11-05 10:58:52,744 - chatgpt_api.api - INFO - SQL query executed successfully


CSV outputs match perfectly.
i =  12


In [26]:
if i<len(llm_query):
    results[i]=(str(i)+'. '+ str(compare_csv.compare_csv(ground_truth_query[i],llm_query[i])))
i+=1 #13
print("i = ", i)

2024-11-05 10:58:52,753 - compare_csv - INFO - Executing LLM query
2024-11-05 10:58:52,755 - urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): localhost:8000
2024-11-05 10:59:07,554 - urllib3.connectionpool - DEBUG - http://localhost:8000 "POST /query HTTP/11" 200 623
2024-11-05 10:59:07,555 - compare_csv - INFO - Executing grouth truth query...
2024-11-05 10:59:07,555 - chatgpt_api.api - DEBUG - Executing SQL query: WITH TopFunds AS (
        SELECT SERIES_NAME, ACCESSION_NUMBER
        FROM FUND_REPORTED_INFO
        ORDER BY TOTAL_ASSETS DESC
        LIMIT 10
    ),
    AssetDistribution AS (
        SELECT H.ASSET_CAT, COUNT(*) AS Category_Count
        FROM FUND_REPORTED_HOLDING H
        JOIN TopFunds T ON H.ACCESSION_NUMBER = T.ACCESSION_NUMBER
        GROUP BY H.ASSET_CAT
    )
    SELECT ASSET_CAT, Category_Count
    FROM AssetDistribution
    ORDER BY Category_Count DESC;
2024-11-05 10:59:07,571 - chatgpt_api.api - INFO - SQL query executed successfully


CSV outputs match perfectly.
i =  13


In [27]:
if i<len(llm_query):
    results[i]=(str(i)+'. '+ str(compare_csv.compare_csv(ground_truth_query[i],llm_query[i])))
i+=1 #14
print("i = ", i)

2024-11-05 10:59:07,580 - compare_csv - INFO - Executing LLM query
2024-11-05 10:59:07,584 - urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): localhost:8000
2024-11-05 10:59:19,599 - urllib3.connectionpool - DEBUG - http://localhost:8000 "POST /query HTTP/11" 200 1272
2024-11-05 10:59:19,600 - compare_csv - INFO - Executing grouth truth query...
2024-11-05 10:59:19,600 - chatgpt_api.api - DEBUG - Executing SQL query: WITH AvgMonthlyReturn AS (
       SELECT ACCESSION_NUMBER, 
              (MONTHLY_TOTAL_RETURN1 + MONTHLY_TOTAL_RETURN2 + MONTHLY_TOTAL_RETURN3) / 3.0 AS Avg_Return
       FROM MONTHLY_TOTAL_RETURN
   )
   SELECT F.SERIES_NAME, A.ACCESSION_NUMBER, A.Avg_Return
   FROM AvgMonthlyReturn A
   JOIN FUND_REPORTED_INFO F ON A.ACCESSION_NUMBER = F.ACCESSION_NUMBER
   ORDER BY A.Avg_Return DESC
   LIMIT 10;
2024-11-05 10:59:19,619 - chatgpt_api.api - INFO - SQL query executed successfully


CSV outputs match perfectly.
i =  14


In [28]:
if i<len(llm_query):
    results[i]=(str(i)+'. '+ str(compare_csv.compare_csv(ground_truth_query[i],llm_query[i])))
i+=1 #15
print("i = ", i)

2024-11-05 10:59:19,629 - compare_csv - INFO - Executing LLM query
2024-11-05 10:59:19,632 - urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): localhost:8000
2024-11-05 10:59:31,160 - urllib3.connectionpool - DEBUG - http://localhost:8000 "POST /query HTTP/11" 200 831
2024-11-05 10:59:31,162 - compare_csv - INFO - Executing grouth truth query...
2024-11-05 10:59:31,162 - chatgpt_api.api - DEBUG - Executing SQL query: WITH TopPerformingFunds AS (
        SELECT 
            ACCESSION_NUMBER, 
            (MONTHLY_TOTAL_RETURN1 + MONTHLY_TOTAL_RETURN2 + MONTHLY_TOTAL_RETURN3) / 3.0 AS Avg_Return
        FROM 
            MONTHLY_TOTAL_RETURN
        ORDER BY 
            Avg_Return DESC
        LIMIT 5
    )
    SELECT 
        FR.SERIES_NAME, 
        FR.NET_ASSETS, 
        TP.Avg_Return
    FROM 
        TopPerformingFunds TP
    JOIN 
        FUND_REPORTED_INFO FR ON TP.ACCESSION_NUMBER = FR.ACCESSION_NUMBER;
2024-11-05 10:59:31,173 - chatgpt_api.api - INFO - SQL que

CSV outputs match perfectly.
i =  15


In [29]:
if i<len(llm_query):
    results[i]=(str(i)+'. '+ str(compare_csv.compare_csv(ground_truth_query[i],llm_query[i])))
i+=1 #16
print("i = ", i)

2024-11-05 10:59:31,183 - compare_csv - INFO - Executing LLM query
2024-11-05 10:59:31,186 - urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): localhost:8000
2024-11-05 10:59:44,008 - urllib3.connectionpool - DEBUG - http://localhost:8000 "POST /query HTTP/11" 200 460
2024-11-05 10:59:44,009 - compare_csv - INFO - Executing grouth truth query...
2024-11-05 10:59:44,009 - chatgpt_api.api - DEBUG - Executing SQL query: WITH LatestReturns AS (
        SELECT 
            M.ACCESSION_NUMBER, 
            M.MONTHLY_TOTAL_RETURN1
        FROM 
            MONTHLY_TOTAL_RETURN M
        JOIN 
            SUBMISSION S ON M.ACCESSION_NUMBER = S.ACCESSION_NUMBER
        WHERE 
            S.REPORT_DATE = (SELECT MAX(REPORT_DATE) FROM SUBMISSION)
    )
    SELECT 
        AVG(MONTHLY_TOTAL_RETURN1) AS Average_Return
    FROM 
        LatestReturns;
2024-11-05 10:59:44,025 - chatgpt_api.api - INFO - SQL query executed successfully


CSV outputs match perfectly.
i =  16


In [30]:
if i<len(llm_query):
    results[i]=(str(i)+'. '+ str(compare_csv.compare_csv(ground_truth_query[i],llm_query[i])))
i+=1 #17
print("i = ", i)

2024-11-05 10:59:44,034 - compare_csv - INFO - Executing LLM query
2024-11-05 10:59:44,036 - urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): localhost:8000
2024-11-05 10:59:58,788 - urllib3.connectionpool - DEBUG - http://localhost:8000 "POST /query HTTP/11" 200 1425
2024-11-05 10:59:58,789 - compare_csv - INFO - Executing grouth truth query...
2024-11-05 10:59:58,789 - chatgpt_api.api - DEBUG - Executing SQL query: WITH InterestRiskScores AS (
        SELECT 
            IR.ACCESSION_NUMBER, 
            -- Calculating composite risk score by summing absolute values of DV01 and DV100 columns
            (ABS(CAST(IR.INTRST_RATE_CHANGE_3MON_DV01 AS FLOAT)) +
            ABS(CAST(IR.INTRST_RATE_CHANGE_1YR_DV01 AS FLOAT)) +
            ABS(CAST(IR.INTRST_RATE_CHANGE_5YR_DV01 AS FLOAT)) +
            ABS(CAST(IR.INTRST_RATE_CHANGE_10YR_DV01 AS FLOAT)) +
            ABS(CAST(IR.INTRST_RATE_CHANGE_30YR_DV01 AS FLOAT)) +
            ABS(CAST(IR.INTRST_RATE_CHANGE_3MON_DV10

CSV outputs match perfectly.
i =  17


In [31]:
if i<len(llm_query):
    results[i]=(str(i)+'. '+ str(compare_csv.compare_csv(ground_truth_query[i],llm_query[i])))
i+=1 #18
print("i = ", i)

2024-11-05 10:59:58,818 - compare_csv - INFO - Executing LLM query
2024-11-05 10:59:58,821 - urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): localhost:8000
2024-11-05 11:00:14,980 - urllib3.connectionpool - DEBUG - http://localhost:8000 "POST /query HTTP/11" 200 178661
2024-11-05 11:00:14,981 - compare_csv - INFO - Executing grouth truth query...
2024-11-05 11:00:14,982 - chatgpt_api.api - DEBUG - Executing SQL query: WITH PortfolioComposition AS (
    SELECT 
        ACCESSION_NUMBER, 
        ASSET_CAT, 
        SUM(CAST(CURRENCY_VALUE AS FLOAT)) AS Total_Value
    FROM 
        FUND_REPORTED_HOLDING
    GROUP BY 
        ACCESSION_NUMBER, 
        ASSET_CAT
    )
    SELECT 
        F.SERIES_NAME, 
        PC.ASSET_CAT, 
        PC.Total_Value
    FROM 
        PortfolioComposition PC
    JOIN 
        FUND_REPORTED_INFO F ON PC.ACCESSION_NUMBER = F.ACCESSION_NUMBER
    ORDER BY 
        F.SERIES_NAME, 
        PC.Total_Value DESC;
2024-11-05 11:00:15,009 - chatgp

CSV outputs match perfectly.
i =  18


In [32]:
if i<len(llm_query):
    results[i]=(str(i)+'. '+ str(compare_csv.compare_csv(ground_truth_query[i],llm_query[i])))
i+=1 #19
print("i = ", i)

2024-11-05 11:00:15,029 - compare_csv - INFO - Executing LLM query
2024-11-05 11:00:15,031 - urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): localhost:8000
2024-11-05 11:00:24,186 - urllib3.connectionpool - DEBUG - http://localhost:8000 "POST /query HTTP/11" 200 378
2024-11-05 11:00:24,187 - compare_csv - INFO - Executing grouth truth query...
2024-11-05 11:00:24,187 - chatgpt_api.api - DEBUG - Executing SQL query: WITH AssetCounts AS (
        SELECT ASSET_CAT, COUNT(*) AS Count
        FROM FUND_REPORTED_HOLDING
        GROUP BY ASSET_CAT
    )
    SELECT ASSET_CAT, Count
    FROM AssetCounts
    ORDER BY Count DESC
    LIMIT 5;
2024-11-05 11:00:24,194 - chatgpt_api.api - INFO - SQL query executed successfully


CSV outputs match perfectly.
i =  19


In [33]:
if i<len(llm_query):
    results[i]=(str(i)+'. '+ str(compare_csv.compare_csv(ground_truth_query[i],llm_query[i])))
i+=1 #20
print("i = ", i)

2024-11-05 11:00:24,202 - compare_csv - INFO - Executing LLM query
2024-11-05 11:00:24,205 - urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): localhost:8000
2024-11-05 11:00:35,692 - urllib3.connectionpool - DEBUG - http://localhost:8000 "POST /query HTTP/11" 200 951
2024-11-05 11:00:35,693 - compare_csv - INFO - Executing grouth truth query...
2024-11-05 11:00:35,693 - chatgpt_api.api - DEBUG - Executing SQL query: WITH AssetChanges AS (
       SELECT F.ACCESSION_NUMBER, F.SERIES_NAME, S.REPORT_DATE, F.NET_ASSETS,
              LAG(F.NET_ASSETS, 1) OVER (PARTITION BY F.SERIES_NAME ORDER BY S.REPORT_DATE) AS Previous_Period_Assets
       FROM FUND_REPORTED_INFO F
       JOIN SUBMISSION S ON F.ACCESSION_NUMBER = S.ACCESSION_NUMBER
   )
   SELECT DISTINCT AC.SERIES_NAME
   FROM AssetChanges AC
   WHERE AC.NET_ASSETS < AC.Previous_Period_Assets
     AND AC.Previous_Period_Assets IS NOT NULL;
2024-11-05 11:00:35,711 - chatgpt_api.api - INFO - SQL query executed successful

Mismatch found.
i =  20


In [34]:
for i in results:
    print(i)

0. True
1. False
2. True
3. True
4. True
5. False
6. True
7. True
8. True
9. True
10. True
11. True
12. True
13. True
14. True
15. True
16. True
17. True
18. True
19. False
